In [1]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime
import os

In [2]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [3]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str}, parse_dates=['period_date'])

In [4]:
pp_dep.period_date.unique()

array(['2020-12-31T00:00:00.000000000', '2021-05-31T00:00:00.000000000',
       '2021-01-31T00:00:00.000000000', '2021-02-28T00:00:00.000000000',
       '2021-03-31T00:00:00.000000000', '2021-04-30T00:00:00.000000000',
       '2020-11-30T00:00:00.000000000', '2020-09-30T00:00:00.000000000',
       '2020-07-31T00:00:00.000000000', '2020-08-31T00:00:00.000000000',
       '2020-10-31T00:00:00.000000000', '2021-06-30T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [7]:
corres = {
    "Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE" : 
        ['Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE'],
    "Nombre de bonus écologiques et de primes à la conversion versés " : 
        ['Nombre de primes à la conversion - DVP',
        'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE'],
    "Nombre de projets industriels soutenus" : [ 'Nombre d’entreprises - BPI',
                                                'Nombre d’entreprises bénéficiaires - PIT', 
                                                'Nombre d’entreprises bénéficiaires - SAC'],
    "Nombre de chèques export et chèques VIE (volontariat international en entreprise" : 
        ['Nombre d’entreprises bénéficiares - SBF'],
    "Nombre de contrats d’apprentissage ayant bénéficié de l’aide exceptionnelle " : 
        ['Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP'],
    "Nombre d’aides versées pour l’embauche des jeunes" : 
        ["Nombre d'aides à l'embauche des jeunes - PEJ"],
    "Nombre de jeunes entrés en CIE (contrats initiative emploi) et PEC (parcours emploi compétences) " : 
        ['Entrées de jeunes en CIE - CIE', 'Entrées de jeunes en PEC - PEC'], 
    "Renforcement des subventions de Business France (chèque export, chèque VIE)" : 
        ['Nombre d’entreprises bénéficiaires - SBF']
}

In [8]:
pd.DataFrame.from_dict(corres, orient='index').to_csv("correspondances.csv")

In [9]:
# On récupère la liste des indicateurs à afficher
list_indicateurs = list(itertools.chain.from_iterable(corres.values()))
len(list_indicateurs)

12

In [10]:
pp_dep_indics = pp_dep[pp_dep['indicateur'].isin(list_indicateurs)].reset_index(drop=True)
pp_dep_indics.drop('Unnamed: 0', axis=1, inplace=True)
pp_dep_indics.shape

(8514, 14)

In [11]:
# Ne garder que la date la plus récente pour chaque indicateur
def get_latest_date_for_indic_and_dep(indicateur, dep):
    df = pp_dep_indics[(pp_dep_indics['indicateur'] == indicateur) & 
                          (pp_dep_indics['dep'] == dep)]
    return df['period_date'].max()


pp_dep_to_export = pp_dep_indics.loc[pp_dep_indics.apply(lambda x: get_latest_date_for_indic_and_dep(x['indicateur'], x['dep']) == x['period_date'], axis=1)]
pp_dep_to_export.reset_index(drop=True, inplace=True)
pp_dep_to_export

,indicateur,state_id,period_date,period_month_tri,Date,valeur,departement,mesure,short_indic,short_mesure,dep,reg,libelle,region
0,Nombre de bonus octroyés à des véhicules élect...,Valeur Actuelle,2021-05-31,May,Mai 2021,3224.0,Seine-et-Marne,Bonus écologique,Nombre de bonus octroyés à des véhicules élect...,Bonus écologique,77,11,Seine-et-Marne,Île-de-France
1,Nombre de bonus octroyés à des véhicules élect...,Valeur Actuelle,2021-05-31,May,Mai 2021,1262.0,Savoie,Bonus écologique,Nombre de bonus octroyés à des véhicules élect...,Bonus écologique,73,84,Savoie,Auvergne-Rhône-Alpes
2,Nombre de bonus octroyés à des véhicules élect...,Valeur Actuelle,2021-05-31,May,Mai 2021,1454.0,Drôme,Bonus écologique,Nombre de bonus octroyés à des véhicules élect...,Bonus écologique,26,84,Drôme,Auvergne-Rhône-Alpes
3,Nombre de bonus octroyés à des véhicules élect...,Valeur Actuelle,2021-05-31,May,Mai 2021,7742.0,Paris,Bonus écologique,Nombre de bonus octroyés à des véhicules élect...,Bonus écologique,75,11,Paris,Île-de-France
4,Nombre de bonus octroyés à des véhicules élect...,Valeur Actuelle,2021-05-31,May,Mai 2021,243.0,Haute-Marne,Bonus écologique,Nombre de bonus octroyés à des véhicules élect...,Bonus écologique,52,44,Haute-Marne,Grand Est
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,Nombre d'aides à l'embauche des jeunes - PEJ,Valeur Actuelle,2021-05-31,May,Mai 2021,10704.0,Haute-Garonne,Prime à l'embauche des jeunes,Nombre d'aides à l'embauche des jeunes,Prime à l'embauche des jeunes,31,76,Haute-Garonne,Occitanie
1197,Nombre d'aides à l'embauche des jeunes - PEJ,Valeur Actuelle,2021-05-31,May,Mai 2021,2381.0,Vienne,Prime à l'embauche des jeunes,Nombre d'aides à l'embauche des jeunes,Prime à l'embauche des jeunes,86,75,Vienne,Nouvelle-Aquitaine
1198,Nombre d'aides à l'embauche des jeunes - PEJ,Valeur Actuelle,2021-05-31,May,Mai 2021,6024.0,Seine-Maritime,Prime à l'embauche des jeunes,Nombre d'aides à l'embauche des jeunes,Prime à l'embauche des jeunes,76,28,Seine-Maritime,Normandie
1199,Nombre d'aides à l'embauche des jeunes - PEJ,Valeur Actuelle,2021-05-31,May,Mai 2021,1490.0,Aveyron,Prime à l'embauche des jeunes,Nombre d'aides à l'embauche des jeunes,Prime à l'embauche des jeunes,12,76,Aveyron,Occitanie


In [12]:
#export = pd.pivot_table(pp_dep_to_export, columns=['dep'], index=['mesure', 'indicateur'])
#export.columns = export.columns.get_level_values(1)
#export['National'] = export.sum(axis=1)
#export.reset_index(drop=False, inplace=True)
#export.set_index('indicateur', inplace=True)

In [13]:
export = pd.pivot_table(pp_dep_to_export, columns=['dep', 'reg', 'departement', 'region'],
                        index=['mesure', 'indicateur'], values="valeur")
#export.columns = export.columns.get_level_values(1)
#export['National'] = export.sum(axis=1)
export.reset_index(inplace=True)
export["index"] = export["mesure"] + "/" + export["indicateur"]
export.set_index('index', inplace=True)
export.drop(columns=["mesure", "indicateur"], inplace=True)
export = export.T
#export.fillna(0, inplace=True)
export = export.loc[~export.index.isin(["00"], level=1)]

/home/mquentin-adc/.local/lib/python3.6/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [18]:
export["Nombre de bonus écologiques et de primes à la conversion versés"] = \
    export["Prime à la conversion des véhicules légers/Nombre de primes à la conversion - DVP"] + \
    export["Bonus écologique/Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE"]

export["Nombre de projets industriels soutenus"] = (
    export["Modernisation des filières automobiles et aéronautiques/Nombre d’entreprises - BPI"] +
    export["Relocalisation : soutien aux projets industriels dans les territoires/Nombre d’entreprises bénéficiaires - PIT"] +
    export["Relocalisation : sécurisation des approvisionnements critiques/Nombre d’entreprises bénéficiaires - SAC"])

export["Nombre de jeunes entrés en CIE et PEC"] = (
    export["PEC jeunes/Entrées de jeunes en PEC - PEC"] +
    export["CIE jeunes/Entrées de jeunes en CIE - CIE"]

)
export

,,,index,Apprentissage/Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP,Bonus écologique/Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE,CIE jeunes/Entrées de jeunes en CIE - CIE,Modernisation des filières automobiles et aéronautiques/Nombre d’entreprises - BPI,PEC jeunes/Entrées de jeunes en PEC - PEC,Prime à l'embauche des jeunes/Nombre d'aides à l'embauche des jeunes - PEJ,Prime à la conversion des véhicules légers/Nombre de primes à la conversion - DVP,Relocalisation : soutien aux projets industriels dans les territoires/Nombre d’entreprises bénéficiaires - PIT,Relocalisation : sécurisation des approvisionnements critiques/Nombre d’entreprises bénéficiaires - SAC,"Renforcement des subventions de Business France (chèque export, chèque VIE)/Nombre d’entreprises bénéficiaires - SBF",Rénovation bâtiments Etat/Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE,Nombre de bonus écologiques et de primes à la conversion versés,Nombre de projets industriels soutenus,Nombre de jeunes entrés en CIE et PEC
dep,reg,departement,region,,,,,,,,,,,,,,
01,84,Ain,Auvergne-Rhône-Alpes,3886.0,1928.0,111.0,10.0,91.0,3434.0,1813.0,7.0,7.0,30.0,7.0,3741.0,24.0,202.0
02,32,Aisne,Hauts-de-France,2433.0,1253.0,216.0,6.0,206.0,1668.0,1490.0,5.0,3.0,17.0,13.0,2743.0,14.0,422.0
03,84,Allier,Auvergne-Rhône-Alpes,1794.0,592.0,98.0,4.0,91.0,1561.0,1106.0,2.0,1.0,18.0,6.0,1698.0,7.0,189.0
04,93,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,1028.0,456.0,31.0,0.0,46.0,583.0,678.0,7.0,0.0,13.0,4.0,1134.0,7.0,77.0
05,93,Hautes-Alpes,Provence-Alpes-Côte d'Azur,781.0,371.0,27.0,2.0,31.0,564.0,547.0,3.0,3.0,3.0,11.0,918.0,8.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,984,Terres australes et antarctiques françaises,Terres australes et antarctiques françaises,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
986,COM,Wallis-et-Futuna,Collectivités d’outre-mer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
987,COM,Polynésie française,Collectivités d’outre-mer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.0


In [19]:
mkdir_ifnotexist('exports')

### Rénovation bâtiments états

In [27]:
tcsv = export["Rénovation bâtiments Etat/Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE"]
tcsv.to_csv("exports/2_renovation_batiment_etat.csv", sep=";")

### Bonus écologique

In [21]:
tcsv = export["Nombre de bonus écologiques et de primes à la conversion versés"]
tcsv.to_csv("exports/3_bonus_ecologique.csv", sep=";")

### Soutien industriel 

In [22]:
tcsv = export["Nombre de projets industriels soutenus"]
tcsv.to_csv("exports/4_soutien_industriel.csv", sep=";")

### Soutien Business France

In [23]:
tcsv = export["Renforcement des subventions de Business France (chèque export, chèque VIE)/Nombre d’entreprises bénéficiaires - SBF"]
tcsv.to_csv("exports/5_cheque_export_vie_sbf.csv", sep=";")

### Apprentissage

In [24]:
tcsv = export["Apprentissage/Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP"]
tcsv.to_csv("exports/6_apprentissage.csv", sep=";")

### Emploi Jeune

In [25]:
tcsv = export["Prime à l'embauche des jeunes/Nombre d'aides à l'embauche des jeunes - PEJ"]
tcsv.to_csv("exports/7_embauche_jeunes.csv", sep=";")

### CIE/PEC

In [26]:
tcsv = export["Nombre de jeunes entrés en CIE et PEC"]
tcsv.to_csv("exports/8_cie_pec.csv", sep=";")